# Topological skeleton analysis

In [1]:
import napari_toska as nts
from skimage.morphology import skeletonize
from skimage.data import binary_blobs
from skimage.measure import label
import numpy as np
import napari

In [2]:
labels = label(binary_blobs(seed=0))

In [3]:
labeled_skeletons = nts.generate_labeled_skeletonization(labels)
labeled_skeletons.max()

15

## Generating skeletons

before diving into actually parsing skeletons and measureing their parameters, let's first summarize a bit of semantics:

- **Labeled skeleton**: An image of skeletonized object, where every skeleton carries a unique label.
- **Parsed skeletons**: Postprocessed, labeled skeletons with the pixel value indicating whether a pixel is an endpoint, a branching point or a branch. It can, and at this point should be, derived for a single skeleton.
- **Labeled branches**: A skeleton where every branch within a single skeleton carries a unique label.

### Intermediate results
- **Adjacency matrix**: A matrix of size $n \times m$, where $n$ is the number of nodes and $m$ is the number of edges. The matrix is binary, with $1$ indicating that the node is connected to the edge.
- **Spine image**: The spine of a skeleton is defined as the longest simple path between two endpoints of a single skeleton. The image retains the branch labels of edges contributing to the spine.

### Network analysis
- **Coarse grained**: A dataframe with features for each skeleton, where each row is a skeleton and each column is a feature.
- **Fine grained**: A dataframe with features for each branch and node, where each row is a branch or node and each column is a feature. This is done *per skeleton*.

## Parse skeletons

In [4]:
parsed_skeletons_single = nts.parse_single_skeleton(labeled_skeletons, label=2, neighborhood='n8')
parsed_skeletons_all = nts.parse_all_skeletons(labeled_skeletons, neighborhood='n8')

In [5]:
viewer = napari.Viewer()

In [6]:
viewer.add_labels(labels, name='labels')
viewer.add_labels(parsed_skeletons_all)
viewer.add_labels(parsed_skeletons_single)
viewer.add_labels(labeled_skeletons)

<Labels layer 'labeled_skeletons' at 0x28d39cc89a0>

In [7]:
adjacency_matrix = nts.create_adjacency_matrix(parsed_skeletons_single, neighborhood='n8')
graph = nts.convert_adjacency_matrix_to_graph(adjacency_matrix)

In [8]:
labeled_branches_single = nts.label_branches(parsed_skeletons_single, labeled_skeletons, neighborhood='n8')
viewer.add_labels(labeled_branches_single)

<Labels layer 'labeled_branches_single' at 0x28d33f0e640>

In [9]:
spine = nts.create_spine_image(adjacency_matrix=adjacency_matrix,
                               labeled_branches=labeled_branches_single)
viewer.add_labels(spine)

<Labels layer 'spine' at 0x28d3b303310>

## Measurements

In [10]:
features = nts.analyze_skeletons(
    labeled_skeletons=labeled_skeletons,
    parsed_skeletons=parsed_skeletons_all)
features

C:\Users\johamuel\Documents\BiAPoL\Projects\napari-toska\src\napari_toska\_measurements.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], axis=0)
C:\Users\johamuel\Documents\BiAPoL\Projects\napari-toska\src\napari_toska\_measurements.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], axis=0)


,skeleton_id,n_endpoints,n_branch_points,n_nodes,n_branches,spine_length,n_cycle_basis,n_possible_undirected_cycles,label
0,1,2,0,2,1,1.0,0,0,1
0,2,11,13,24,25,13.0,2,3,2
0,3,2,0,2,1,1.0,0,0,3
0,4,2,0,2,1,1.0,0,0,4
0,5,2,0,2,1,1.0,0,0,5
0,6,2,0,2,1,1.0,0,0,6
0,7,6,4,10,9,5.0,0,0,7
0,8,2,0,2,1,1.0,0,0,8
0,9,2,0,2,1,1.0,0,0,9
0,10,0,0,0,0,NaN,0,0,10


In [11]:
nts.analyze_single_skeleton_network(parsed_skeletons_single, neighborhood='n8')

,label,component_type,degree,weight,node_1,node_2,node_labels
0,1,node,3.0,NaN,NaN,NaN,0.0
1,2,node,3.0,NaN,NaN,NaN,13.0
2,3,node,3.0,NaN,NaN,NaN,1.0
3,4,node,3.0,NaN,NaN,NaN,14.0
4,5,node,3.0,NaN,NaN,NaN,15.0
5,6,node,3.0,NaN,NaN,NaN,2.0
6,7,node,3.0,NaN,NaN,NaN,3.0
7,8,node,3.0,NaN,NaN,NaN,6.0
8,9,node,3.0,NaN,NaN,NaN,16.0
9,10,node,3.0,NaN,NaN,NaN,4.0
